# Hanabi game simulation: Results analysis

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os

In [2]:
results_parent_path = "/home/nmontes/OneDrive/Documentos/PhD/hanabi-results"
num_players = 2
slots_per_player = 5 if num_players == 2 or num_players == 3 else 4

In [3]:
path = "{}/{}_players".format(results_parent_path, num_players)
seeds = [int(folder.split('_')[-1]) for folder in os.listdir(path)]
simulation_runs = len(seeds)
print("For {} players, {} simulations have been run".format(num_players, simulation_runs))

max_seed = 500
missing_seeds = [s for s in range(max_seed) if not s in seeds]
print("The following seeds are missing:", *missing_seeds)

For 2 players, 482 simulations have been run
The following seeds are missing: 57 66 68 181 187 189 194 212 229 231 236 239 248 266 271 294 474 489


## Analysis of one game

In [4]:
# analyze one game
seed = 15
evolution_file = "{}/{}_players/results_{}_{}/evolution.csv".format(results_parent_path, num_players, num_players, seed)
evolution = pd.read_csv(evolution_file, sep=';')
print(evolution.head())

# find the final score and the total number of hints
final_score = evolution["score"].iloc[-1]
total_hints = evolution["hints"].iloc[-1]
efficiency = final_score / total_hints

print("Score: {}".format(final_score))
print("Hints: {}".format(total_hints))
print("Efficiency: {:.2f}".format(efficiency))

   move player action_functor             actions_args  hints  score
0     1  alice      give_hint   bob, color, white, [2]      1      0
1     2    bob      give_hint  alice, color, blue, [1]      2      0
2     3  alice      give_hint        bob, rank, 1, [2]      3      0
3     4    bob      play_card                        2      3      1
4     5  alice   discard_card                        2      3      1
Score: 15
Hints: 33
Efficiency: 0.45


In [6]:
# find the game moves where hints were given
hint_moves = evolution.loc[evolution["action_functor"] == "give_hint"]

# get the n-th argument from string s (arguments are comma-separated)
def get_arg(s, n, sep):
    l = [a.strip() for a in s.split(sep)]
    return l[n]

receiver = hint_moves["actions_args"].apply(lambda s: get_arg(s, 0, ','))
hint_moves.insert(hint_moves.shape[1], "receiver", receiver)
print(hint_moves.head())

   move player action_functor                actions_args  hints  score  \
0     1  alice      give_hint      bob, color, white, [2]      1      0   
1     2    bob      give_hint     alice, color, blue, [1]      2      0   
2     3  alice      give_hint           bob, rank, 1, [2]      3      0   
5     6    bob      give_hint         alice, rank, 1, [1]      4      1   
9    10    bob      give_hint  alice, color, white, [1,3]      5      2   

  receiver  
0      bob  
1    alice  
2      bob  
5    alice  
9    alice  


In [7]:
n = 0
m = hint_moves["move"].iloc[n]
print(hint_moves.iloc[n])

# retrieve receiver of the hint
rec = hint_moves.loc[hint_moves["move"] == m, "receiver"].iloc[0]

move                                   1
player                             alice
action_functor                 give_hint
actions_args      bob, color, white, [2]
hints                                  1
score                                  0
receiver                             bob
Name: 0, dtype: object


In [18]:
pre_action_file = "{}/results_{}_{}/{}_{}_false.csv".format(path, num_players, seed, rec, m)
post_action_file = "{}/results_{}_{}/{}_{}_false.csv".format(path, num_players, seed, rec, m+1)
post_explanation_file = "{}/results_{}_{}/{}_{}_true.csv".format(path, num_players, seed, rec, m+1)

pre_action_distribution = pd.read_csv(pre_action_file, sep=';')
post_action_distribution = pd.read_csv(post_action_file, sep=';')

def compute_probability(df):
    den = df.groupby("slot")["num"].sum()
    for slot, n in den.iteritems():
        assert n > 0, "probability distribution for slot {} is ill-defined".format(slot)
    prob = df.apply(lambda x: x.num/den[x.slot], axis=1)
    df.insert(df.shape[1], "prob", prob)

compute_probability(pre_action_distribution)
compute_probability(post_action_distribution)

print(pre_action_distribution.head())
print(post_action_distribution.head())

   slot color  rank  num      prob
0     3   red     4    2  0.044444
1     3   red     5    1  0.022222
2     3   red     1    3  0.066667
3     3   red     2    2  0.044444
4     3   red     3    2  0.044444
   slot  color  rank  num  prob
0     5  white     4    0   0.0
1     5  white     5    0   0.0
2     5  white     1    0   0.0
3     5  white     2    0   0.0
4     5  white     3    0   0.0


In [28]:
from itertools import product

def kullback_leibler_distance(Q, P, slot, base=10):
    colors = ["red", "green", "white", "blue", "yellow"]
    ranks = [1, 2, 3, 4, 5]
    D_kl = 0
    for c, r in product(colors, ranks):
        Q_prob = Q.loc[(Q["slot"] == slot) & (Q["color"] == c) & (Q["rank"] == r), "prob"].iloc[0]
        P_prob = P.loc[(P["slot"] == slot) & (P["color"] == c) & (P["rank"] == r), "prob"].iloc[0]
        if np.isclose(P_prob, 0, atol=1.0E-9):
            continue
        D_kl += P_prob * math.log(P_prob/Q_prob, base)
    return D_kl

explicit_info_mre = {}
for s in range(1, slots_per_player+1):
    d = kullback_leibler_distance(pre_action_distribution, post_action_distribution, s)
    explicit_info_mre[s] = d

print("MRE using explicit information:")
print("-------------------------------")
for s, mre in explicit_info_mre.items():
    print("Slot {} -- D_KL = {:.5f}".format(s, mre))

MRE using explicit information:
-------------------------------
Slot 1 -- D_KL = 0.09691
Slot 2 -- D_KL = 0.69897
Slot 3 -- D_KL = 0.09691
Slot 4 -- D_KL = 0.09691
Slot 5 -- D_KL = 0.09691


In [33]:
post_explanation_distribution = pd.read_csv(post_explanation_file, sep=';')
# compute_probability(post_explanation_distribution)
print(post_explanation_distribution.head())
post_explanation_distribution.loc[post_explanation_distribution["slot"] == 2]

   slot  color  rank  num
0     2  white     3    0
1     2  white     2    0
2     2  white     1    0
3     2  white     5    0
4     2  white     4    0


,slot,color,rank,num
0,2,white,3,0
1,2,white,2,0
2,2,white,1,0
3,2,white,5,0
4,2,white,4,0
20,2,red,4,0
21,2,red,5,0
22,2,red,1,0
23,2,red,2,0
24,2,red,3,0
